In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import datetime
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import sys

sys.path.append("..")

import slicer
from slicer import models

N_MINIBATCH = 64
N_EPOCHS = 20
LR = 0.01
PATIENCE = 3

%matplotlib inline
%load_ext autoreload
%autoreload 2

# Read Data

In [ ]:
X_train = np.load("../data/X_train.npy").reshape((-1, 80, 322, 1))
Y_train = np.load("../data/Y_train.npy")
W_train = np.load("../data/W_train.npy")

X_test = np.load("../data/X_test.npy").reshape((-1, 80, 322, 1))
Y_test = np.load("../data/Y_test.npy")
W_test = np.load("../data/W_test.npy")

In [ ]:
max_print = 3
for idx in np.where(Y_train == 1)[0][1004:1004 + max_print]:
    plt.figure()
    plt.imshow(X_train[idx].reshape(80, 322), aspect="auto")

# Train

In [ ]:
model = models.CNN().model

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LR),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.BinaryAccuracy()],
)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=False)
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# keras.utils.plot_model(model, show_shapes=True)

In [ ]:
history = model.fit(
    X_train,
    Y_train,
    batch_size=N_MINIBATCH,
    epochs=N_EPOCHS,
    validation_data=(X_test, Y_test),
    callbacks=[callback, tensorboard_callback],
)

# Inference

In [ ]:
import librosa

from slicer import utils
from slicer.constants import SR
from slicer.constants import HOP_LENGTH
from slicer.constants import PEAK_PICKING_THRES
from slicer.constants import PINK_NOISE_DUR

def do_the_slice(mel, plot=False):
    # Preprocess
    mel = utils.to_db(utils.pad_pink_noise(mel))
    
    # Get patches and do inference
    X = utils.get_mel_patches(mel, 1, 322)
    Y = model.predict(X.reshape((-1, 80, 322, 1)))
    
    # Obtain boundaries
    bounds = utils.peak_picking(Y.flatten(), PEAK_PICKING_THRES)
    
    # Plot results
    if plot:
        # Plot novelty curve
        plt.figure(figsize=(20, 5))
        plt.plot(Y)
        for b in bounds:
            plt.vlines(b, ymin=0, ymax=1, color='red')
    
        # Plot melspec with boundaries
        noise = int(PINK_NOISE_DUR * SR // HOP_LENGTH)
        plt.figure(figsize=(20, 5))
        plt.imshow(mel, aspect="auto")
        for b in bounds:
            plt.vlines(b + noise, ymin=0, ymax=80, color='red')
    
    return bounds
            
    

AUDIO = "../trains.mp3"
# AUDIO = "../paranoid.mp3"
# AUDIO = "../bohemian.mp3"
audio, _ = librosa.load(AUDIO, sr=slicer.constants.SR)

In [ ]:
mel = utils.compute_melspec(audio)
bounds = do_the_slice(mel, plot=True)

In [ ]:
import IPython
c = librosa.clicks(frames=bounds, sr=SR, hop_length=HOP_LENGTH, length=len(audio))
IPython.display.Audio(audio + c, rate=SR)

In [ ]:
model.save('../models/cnn')

# Evaluate

In [ ]:
import mir_eval
import os



with open("../data/names_test.txt", "r") as f:
    test_names = f.readlines()


for name in test_names:
    name = name.replace("\n", "")
    mel = np.load(os.path.join("../melspecs/", name + "-mel.npy"))
    slicer.utils.load_bounds(os.path.join("../har"))
